<a href="https://colab.research.google.com/github/SimonSanfeliu/Algoritmos-y-Estructuras-de-Datos/blob/master/Copy_of_Tarea5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC3001 Otoño 2020 Tarea 5
# Simulación con Colas y Colas de Prioridad

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 22 de junio de 2020 a las 23:59

El objetivo de esta tarea es aprender a escribir programas de simulación sencillos, haciendo uso de colas y de colas de prioridad.

En primer lugar, consideremos una caja a la que van llegando clientes en instantes aleatorios, separados por intervalos de tamaño promedio $t_a$. Cada cliente que llega se pone al final de la cola, espera su turno, y cuando llega a la cabeza de la cola lo empiezan a atender. La atención dura también un tiempo aleatorio de tamaño promedio $t_s$. Cuando completa su atención, sale del sistema, y en la caja se empieza a atender al siguiente cliente de la cola, en caso de que haya alguien.

![colaT5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/colaT5.png?raw=1)

Los tiempos entre llegadas y el tiempo de atención se modelan con distribuciones exponenciales, lo cual es la suposición usual en este tipo de simulaciones.

A continuación veremos un programa que simula este proceso para un número dado de clientes y calcula el largo promedio de la cola y la demora total promedio de los clientes.

El programa lleva un reloj simulado, que comienza en cero y va  avanzando monótonamente, al siguiente instante en que un cliente llega al sistema o al siguiente instante en que un cliente sale del sistema, lo que sea que ocurra primero.

Como hay solo dos tipos posible de eventos de interés, tendremos una variable para cada uno de ellos (``evento_llegada`` y ``evento_salida``), que almacenan los respectivos tiempos. Si en este momento no hay nadie que deba salir del sistema, ``evento_salida`` es ``None``, lo cual ocurre cada vez que la cola de espera queda vacía. En cambio, ``evento_llegada`` solo es ``None``cuando ya han llegado todos los clientes que debían llegar.

Nótese que no es necesario tener generados los tiempos de todas las llegadas futuras, basta con tener el tiempo de la próxima llegada, y cuando ella se cumple se genera la siguiente.

En cada elemento que está en la cola de espera anotamos solo la hora a la que llegó ese cliente, que es lo único que necesitamos saber para poder calcular, en el momento en que sale, cuánto tiempo demoró en el sistema.

Estudie con atención este programa y la bitácora que él genera.

In [ ]:
import numpy as np
class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

def print_list(L):
    for x in L:
        print(x)

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)
    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    c=Cola()
    evento_llegada=np.random.exponential(ta)
    evento_salida=None
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    
    while evento_llegada is not None or evento_salida is not None:
        # Ver si el próximo evento que toca que ocurra es la llegada o la salida de un cliente
        if (evento_salida is None) or (evento_llegada is not None and (evento_llegada<=evento_salida)):
            # Llega un cliente, avanzamos el tiempo simulado a la hora en que llega
            ahora=evento_llegada
            nclientes+=1
            
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => LLEGADA"+
                                "\n\tcola="+str(c.q))
            # El cliente que acaba de llegar se pone a la cola
            c.enq(ahora) # Basta anotar su hora de llegada
            acum_largo_cola+=c.size()
            
            if c.size()==1: # la cola estaba vacía, lo empezamos a atender de inmediato
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
            
            if nclientes<maxclientes: # programamos la hora de llegada del siguiente cliente
                evento_llegada=ahora+np.random.exponential(ta)
            else:
                evento_llegada=None
        else:               
            # Termina de atenderse un cliente, avanzamos el tiempo simulado a la hora en que sale
            ahora=evento_salida
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => SALIDA"+
                                "\n\tcola="+str(c.q))
            tllegada=c.deq() # sacamos al cliente de la cola
            acum_tiempo_en_sistema+=(ahora-tllegada)
            
            if c.is_empty(): # no hay nadie en la cola
                evento_salida=None
            else: # Hay un cliente esperando, empezamos a atenderlo
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
    
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)            

In [ ]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722


In [ ]:
print_list(bitacora)

T=21.25986576184801 evento_llegada=21.25986576184801 evento_salida=None => LLEGADA
	cola=[]
T=78.83677538497456 evento_llegada=78.83677538497456 evento_salida=94.24603167524747 => LLEGADA
	cola=[21.25986576184801]
T=94.24603167524747 evento_llegada=232.71542283099382 evento_salida=94.24603167524747 => SALIDA
	cola=[78.83677538497456, 21.25986576184801]
T=207.7973648436669 evento_llegada=232.71542283099382 evento_salida=207.7973648436669 => SALIDA
	cola=[78.83677538497456]
T=232.71542283099382 evento_llegada=232.71542283099382 evento_salida=None => LLEGADA
	cola=[]
T=256.5855663702635 evento_llegada=265.07595569718336 evento_salida=256.5855663702635 => SALIDA
	cola=[232.71542283099382]
T=265.07595569718336 evento_llegada=265.07595569718336 evento_salida=None => LLEGADA
	cola=[]
T=386.489172340086 evento_llegada=582.4168679105603 evento_salida=386.489172340086 => SALIDA
	cola=[265.07595569718336]
T=582.4168679105603 evento_llegada=582.4168679105603 evento_salida=None => LLEGADA
	cola=[]


## Lo que usted tiene que hacer

### 1. Mejorar este programa introduciendo una cola de prioridad

El programa está escrito sabiendo que puede haber dos tipos de eventos por ocurrir (LLEGADA y SALIDA) y que en todo momento puede haber a lo más solo uno de cada tipo.

En una situación más general, podría haber eventos de muchos tipos programados para ocurrir, en muchos instantes distintos. En ese caso, la implementación presentada es demasiado limitada, y se requiere un mecanismo más general para administrar esa lista de eventos futuros. Una *cola de prioridad* (usando el tiempo como prioridad, y en que el mínimo tiempo equivale a mejor prioridad) es la estructura adecuada, porque en esa lista de eventos futuros siempre vamos extrayendo el primero que debe ocurrir (el mínimo), y durante la ejecución se pueden generar nuevos elementos programados para ocurrir en el futuro (inserciones).

En preparación a poder simular modelos más complejos, lo que usted tiene que hacer es implementar una lista de eventos futuros mediante uan cola de prioridad, en la cual se almacenen pares de la forma

$$
(\text{tiempo},\text{tipo de evento})
$$

y modificar el programa para que utilice esa cola de prioridad en lugar de las variables ``evento_salida`` y ``evento_llegada``. Implemente la cola de prioridad en base al código para heaps que aparece en el apunte.

Sugerencias: Antes del while, encolar el primer evento de llegada, luego en cada iteración sacar un evento de la cola de prioridad y procesarlo, y cuando en el programa original se generaba un evento nuevo y se asignaba a la variable respectiva, ahora hay que insertar un nuevo evento del tipo correspondiente en la cola de prioridad.

El programa resultante debiera ser más simple, porque varias cosas se unifican al verlas de esta manera.

Ejecute ambos programas en modo ``debug`` y asegúrese que en las bitácoras resultantes los eventos que ocurren y sus tiempos sean idénticos.

## 2. Modificar el modelo para introducir un tiempo comprando

A continuación, introduciremos un elemento adicional en el modelo, y usted debe modificar su programa para incluirlo.

Supondremos que una vez que un cliente llega, en lugar de ponerse a la cola de inmediato, pasa un rato en la tienda comprando, y una vez que termina (después de transcurrido un tiempo aleatorio de duración promedio $t_c$), en ese momento recién se pone a la cola para pagar en la caja.

![cola2T5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/cola2T5.png?raw=1)

Observe que, como puede haber muchos clientes en la tienda que aún no terminan de comprar, en la lista de eventos futuros ahora puede haber muchos eventos de un nuevo tipo (FINCOMPRA), lo que justifica el haber introducido la cola de prioridad.

Ejecute su programa en modo ``debug`` agregando un parámetro $t_c=300$ e imprima el largo promedio de la cola, la demora total promedio de los clientes y la bitácora.

## 3. ¿Qué hay que entregar?

Usted debe entregar este mismo archivo, modificado de acuerdo a lo que se pide. Haga todos los cambios necesarios para explicar y documentar adecuadamente su código. No olvide poner su nombre.

## Parte 1

Se agrega la cola de prioridad en forma de Heap, editada de tal manera que funcione con el menor valor posible como mayor prioridad y para que funcione con las tuplas pedidas. Una vez hecho esto, se edita la función "simula" utilizando esta herramienta para eliminar las variables 'evento_llegada' y 'evento_salida'. Todo esto perimitirá que la parte 2 salga más fluida, ya que con estos cambios se puede modificar la función para que acepte más eventos.

In [144]:
import numpy as np
class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

def print_list(L):
    for x in L:
        print(x)

def trepar(a,j):                            #Como los elementos del arreglo ahora son tuplas y el primer elemento de esta tupla define su prioridad,
  while j>=1 and a[j][0]<a[(j-1)//2][0]:    #son estos elementos los que deben ser comparados. 
    (a[j],a[(j-1)//2]) = (a[(j-1)//2],a[j]) #Se cambian los elementos enteros porque su prioridad no cambia el tipo de evento que es.
    j = (j-1)//2                            #Las comparaciones quedan al revés porque a menor tiempo, mayor prioridad.

def hundir(a,j,n):
  while 2*j+1<n:
    k=2*j+1
    if k+1<n and a[k+1][0]<a[k][0]: #Trabajando con el tiempo de las tuplas.
      k+=1
    if a[j][0]<=a[k][0]:
      break
    (a[j],a[k]) = (a[k],a[j])
    j=k

class Heap:
  def __init__(self,maxn=100):
    self.a = np.zeros(maxn, dtype = tuple) #Se modifica el arreglo para trabajar con las tuplas pedidas.
    self.n = 0
  def insert(self,x): #Asumiendo que se inserta otra tupla.
    assert self.n<len(self.a)
    self.a[self.n]=x
    trepar(self.a,self.n)
    self.n += 1
  def extract_max(self):
    x = self.a[0]
    self.n -= 1
    self.a[0]=self.a[self.n]
    hundir(self.a,0,self.n)
    return x
  def imprimir(self):
    print(self.a[0:self.n])

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)
    if debug:
        np.random.seed(1234)

    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    c=Cola() #La cola que tendrá los tiempos de llegada y salida a lo largo del proceso.
    h = Heap() #Se agrega la cola de prioridad pedida.
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug

    ti = np.random.exponential(ta) #Siguiendo el hint, se inserta el primer evento de llegada fuera del while. 
    h.insert((ti, 'llegada'))      #Se inserta dos veces dado el invariante del while, el que saca el máximo del heap, pero luego debe extraerse
    h.insert((ti, 'llegada'))      #este mismo máximo para el ciclo; cosa que por la definción de extract_max estaría extrayendo otro máximo. Así,
                                   #se agrega una copia de este mismo evento para hacer el ciclo de manera correcta.

    while h.extract_max()[1] is not None: #Se verifica que la tupla efectivamente contenga un evento.
        k = h.extract_max() #Se extrae el máximo del heap, es decir, el evento más cercano a ocurrir.
        if k[1] == 'llegada': 
          ahora = k[0] #El tiempo actual es el tiempo de la tupla.
          nclientes += 1 #Llego un cliente.

          if debug:
            bitacora.append('T = '+str(ahora)+' Tipo de evento: '+k[1]) #Bitácora utilizando lo dicho en el foro por el profesor.

          c.enq(ahora) #Se guarda el tiempo actual en la cola y se agrega el tamaño actual de ésta a la variables acum_largo_cola.
          acum_largo_cola+=c.size()

          if c.size() == 1: #Si es que hay sólo un elemento en la cola, sólo hay una persona en ésta; por lo que se atiende de inmediato.
            tf = ahora + np.random.exponential(ts) 
            s = (tf, 'salida') #Al atender al cliente, se genera la tupla con su tiempo de salida y con el evento 'salida'.
            h.insert(s) #Se inserta la tupla al heap para ordenar el proceso de simulación.
            h.insert(s) #Dado el invariante del while, siempre se deben agregar dos veces las tuplas para que funcione correctamente.
          if nclientes < maxclientes: #Si aun pueden haber más clientes, se genera una tupla de llegada para el siguiente cliente.
            ti = ahora + np.random.exponential(ta)
            e = (ti, 'llegada')
            h.insert(e)
            h.insert(e)
          else:
            g = (k[0], 'no más llegadas') #Al llegar a la cantidad máxima de clientes, se crea este evento para que se genere la última salida.
            h.insert(g)
        else:
          ahora = k[0] #El tiempo actual es el tiempo de la tupla.
          if debug:
            bitacora.append('T = '+str(k[0])+ ' Tipo de evento: '+k[1]) #Se guarda la información del evento en la bitácora (de ser pedida).
          
          tllegada = c.deq() #De la cola se saca el tiempo de llegada asociado al tiempo de salida que viene de la tupla de salida.
          acum_tiempo_en_sistema += (ahora-tllegada) #La resta entre tsalida y tllegada da cuánto tiempo estuvo el cliente en el sistema.

          if c.is_empty(): 
            if nclientes == maxclientes: #Si la cola estaba vacía y además se llegó al máximo de clientes, entonces se termina la simulación (ya se
              break                      #resolvieron todos los casos, todos los clientes ya salieron).
            else:  #Si es que aun quedan clientes por llegar, se pasa al siguiente elemento del heap, ya que la cola puede quedar vacía pero aún
              pass #pueden llegar clientes.
          else:
            tf = ahora + np.random.exponential(ts) #Si es que aún hay gente en la cola, se crea un evento salida para los que están ahí.
            s = (tf, 'salida')
            h.insert(s)
            h.insert(s)
        
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)

In [145]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722


In [146]:
print_list(bitacora)

T = 21.25986576184801 Tipo de evento: llegada
T = 78.83677538497456 Tipo de evento: llegada
T = 94.24603167524747 Tipo de evento: salida
T = 207.7973648436669 Tipo de evento: salida
T = 232.71542283099382 Tipo de evento: llegada
T = 256.5855663702635 Tipo de evento: salida
T = 265.07595569718336 Tipo de evento: llegada
T = 386.489172340086 Tipo de evento: salida
T = 582.4168679105603 Tipo de evento: llegada
T = 626.7051068662289 Tipo de evento: llegada
T = 696.2190483450556 Tipo de evento: llegada
T = 738.9366623619304 Tipo de evento: salida
T = 811.2505419244258 Tipo de evento: llegada
T = 832.4793149848136 Tipo de evento: salida
T = 857.4938981550002 Tipo de evento: llegada
T = 894.2570294009206 Tipo de evento: salida
T = 895.296837936396 Tipo de evento: salida
T = 927.427158306087 Tipo de evento: llegada
T = 1006.4499708056504 Tipo de evento: salida
T = 1167.1389175826548 Tipo de evento: salida


## Parte 2


Para esta parte, se debe modificar la función "simula" (ahora llamada "simula2") para que acepte más tipos de eventos, haciendo la simulación más realista. Para no tener una lista tan grande de eventos, se agrega el evento genérico "fincompra", que es cuando el cliente dejó de comprar y pasa a la cola para pagar. Para saber cuánto tiempo se demora en comprar, se introduce la variable $t_c$, la que representa el tiempo promedio de compras. Así, con toda esta información se puede modificar la simulación de la forma pedida, utilizando de base el código de la parte 1, el cual está hecho para poder usar el Heap de la forma más óptima para este caso.

In [173]:
import numpy as np
class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

def print_list(L):
    for x in L:
        print(x)

def trepar(a,j):                            #Como los elementos del arreglo ahora son tuplas y el primer elemento de esta tupla define su prioridad,
  while j>=1 and a[j][0]<a[(j-1)//2][0]:    #son estos elementos los que deben ser comparados. 
    (a[j],a[(j-1)//2]) = (a[(j-1)//2],a[j]) #Se cambian los elementos enteros porque su prioridad no cambia el tipo de evento que es.
    j = (j-1)//2                            #Las comparaciones quedan al revés porque a menor tiempo, mayor prioridad.

def hundir(a,j,n):
  while 2*j+1<n:
    k=2*j+1
    if k+1<n and a[k+1][0]<a[k][0]: #Trabajando con el tiempo de las tuplas.
      k+=1
    if a[j][0]<=a[k][0]:
      break
    (a[j],a[k]) = (a[k],a[j])
    j=k

class Heap:
  def __init__(self,maxn=100):
    self.a = np.zeros(maxn, dtype = tuple) #Se modifica el arreglo para trabajar con las tuplas pedidas.
    self.n = 0
  def insert(self,x): #Asumiendo que se inserta otra tupla.
    assert self.n<len(self.a)
    self.a[self.n]=x
    trepar(self.a,self.n)
    self.n += 1
  def extract_max(self):
    x = self.a[0]
    self.n -= 1
    self.a[0]=self.a[self.n]
    hundir(self.a,0,self.n)
    return x
  def imprimir(self):
    print(self.a[0:self.n])

def simula2(maxclientes,ta,ts,tc,debug=False):
    # simula el paso de maxclientes clientes por el sistema
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # tc = tiempo promedio de compra
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)
    if debug:
        np.random.seed(1234)

    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    c=Cola() #La cola que tendrá los tiempos de llegada y salida a lo largo del proceso.
    h = Heap() #Se agrega la cola de prioridad pedida.
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    tllegada = 0 #Variable para sumar todos los tiempos de llegada.
    tsalida = 0 #Variable para sumar todos los tiempos de salida.

    ti = np.random.exponential(ta) #Siguiendo el hint, se inserta el primer evento de llegada fuera del while. 
    h.insert((ti, 'llegada'))      #Se inserta dos veces dado el invariante del while, el que saca el máximo del heap, pero luego debe extraerse
    h.insert((ti, 'llegada'))      #este mismo máximo para el ciclo; cosa que por la definción de extract_max estaría extrayendo otro máximo. Así,
                                   #se agrega una copia de este mismo evento para hacer el ciclo de manera correcta.

    while h.extract_max()[1] is not None: #Se verifica que la tupla efectivamente contenga un evento.
        k = h.extract_max() #Se extrae el máximo del heap, es decir, el evento más cercano a ocurrir.
        if k[1] == 'llegada': 
          ahora = k[0] #El tiempo actual es el tiempo de la tupla.
          nclientes += 1 #Llegó un cliente.
          tllegada += ahora #Se agrega el tiempo de llegada al total de tiempos de llegada.

          tco = ahora + np.random.exponential(tc) #Se agrega el valor que se demora en comprar y se crea el evento 'fincompra'.
          co = (tco, 'fincompra')
          h.insert(co) #Se inserta dos veces en el Heap para mantener la condición del invariante.
          h.insert(co)

          if debug:
            bitacora.append('T = '+str(ahora)+' Tipo de evento: '+k[1]) #Bitácora utilizando lo dicho en el foro por el profesor.

          if nclientes < maxclientes: #Si aun pueden haber más clientes, se genera una tupla de llegada para el siguiente cliente.
            ti = ahora + np.random.exponential(ta)
            e = (ti, 'llegada')
            h.insert(e)
            h.insert(e)
          else:
            g = (k[0], 'fincompra') #Al llegar a la cantidad máxima de clientes, se crea este evento para que se genere la última salida.
            h.insert(g)

          h.imprimir()
          print_list(c.q)
          
        if k[1] == 'fincompra':
          ahora = k[0]

          if debug:
            bitacora.append('T = '+str(k[0])+ ' Tipo de evento: '+k[1]) #Se guarda la información del evento en la bitácora (de ser pedida).

          c.enq(ahora) #Se guarda el tiempo actual en la cola y se agrega el tamaño actual de ésta a la variable acum_largo_cola.
          acum_largo_cola+=c.size()

          if c.size() == 1: #Si es que hay sólo un elemento en la cola, sólo hay una persona en ésta; por lo que se atiende de inmediato.
            tf = ahora + np.random.exponential(ts) 
            s = (tf, 'salida') #Al atender al cliente, se genera la tupla con su tiempo de salida y con el evento 'salida'.
            h.insert(s) #Se inserta la tupla al heap para ordenar el proceso de simulación.
            h.insert(s) #Dado el invariante del while, siempre se deben agregar dos veces las tuplas para que funcione correctamente.
          
          h.imprimir()
          print_list(c.q)

        else:
          ahora = k[0] #El tiempo actual es el tiempo de la tupla.
          tsalida += ahora

          if debug:
            bitacora.append('T = '+str(k[0])+ ' Tipo de evento: '+k[1]) #Se guarda la información del evento en la bitácora (de ser pedida).
          
          c.deq() #De la cola se saca el tiempo de llegada asociado al tiempo de salida que viene de la tupla de salida.

          if c.is_empty(): 
            if nclientes == maxclientes: #Si la cola estaba vacía y además se llegó al máximo de clientes, entonces se termina la simulación (ya se
              break                      #resolvieron todos los casos, todos los clientes ya salieron).
            else:  #Si es que aun quedan clientes por llegar, se pasa al siguiente elemento del heap, ya que la cola puede quedar vacía pero aún
              pass #pueden llegar clientes.
          else:
            tf = ahora + np.random.exponential(ts) #Si es que aún hay gente en la cola, se crea un evento salida para los que están ahí.
            s = (tf, 'salida')
            h.insert(s)
            h.insert(s)
          
        h.imprimir()
        print_list(c.q)

        
    tp = abs(tllegada - tsalida)    
    return (acum_largo_cola/maxclientes,tp/maxclientes,bitacora)

In [174]:
(cola,demora,bitacora)=simula2(10,100,75,300,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

[(78.83677538497456, 'llegada') (78.83677538497456, 'llegada')
 (313.20452941544585, 'fincompra') (313.20452941544585, 'fincompra')]


AssertionError: ignored

In [166]:
print_list(bitacora)

T = 21.25986576184801 Tipo de evento: llegada
T = 21.25986576184801 Tipo de evento: llegada
T = 78.83677538497456 Tipo de evento: llegada
T = 78.83677538497456 Tipo de evento: llegada
T = 230.23855294286716 Tipo de evento: llegada
T = 230.23855294286716 Tipo de evento: llegada
T = 262.5990858090567 Tipo de evento: llegada
T = 262.5990858090567 Tipo de evento: llegada
T = 313.20452941544585 Tipo de evento: fincompra
T = 325.719127099946 Tipo de evento: fincompra
T = 540.4727177230324 Tipo de evento: fincompra
T = 579.9399980224337 Tipo de evento: llegada
T = 579.9399980224337 Tipo de evento: llegada
T = 624.2282369781022 Tipo de evento: llegada
T = 624.2282369781022 Tipo de evento: llegada
T = 739.2597305574725 Tipo de evento: llegada
T = 739.2597305574725 Tipo de evento: llegada
T = 748.2519523806671 Tipo de evento: fincompra
T = 785.503086788047 Tipo de evento: llegada
T = 785.503086788047 Tipo de evento: llegada
T = 832.7700614145824 Tipo de evento: fincompra
T = 855.4363469391337 Ti